In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
import cv2
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Dataset

In [3]:
animals = {0: 'butterfly', 1: 'cat', 2: 'cow', 3: 'dog', 4: 'elephant', 5: 'hen', 6: 'horse', 7: 'sheep', 8: 'spider', 9: 'squirrel'}
rev_animals = {j:i for i,j in animals.items()}
print(rev_animals)

{'butterfly': 0, 'cat': 1, 'cow': 2, 'dog': 3, 'elephant': 4, 'hen': 5, 'horse': 6, 'sheep': 7, 'spider': 8, 'squirrel': 9}


In [4]:
imgx, imgy = 100, 100
bs = 20000

In [5]:
path_to_main_dir = '/home/arjun/Desktop/Datasets/animals'
X,y = [], []
for sub in os.listdir(path_to_main_dir):
    for img in os.listdir(os.path.join(path_to_main_dir,sub))[:1000]:
        image = cv2.imread(os.path.join(os.path.join(path_to_main_dir,sub), img))
        x = cv2.resize(image, (imgx, imgy))
        X.append(x)
        y.append(rev_animals[sub])
len(X)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


10000

In [6]:
X = torch.tensor(X).to(device).float()
y = np.array(y)
y = F.one_hot(torch.tensor(y), num_classes=10).to(device).float() # Super slow :(

/tmp/ipykernel_7264/7877835.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  X = torch.tensor(X).to(device).float()


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=.8)

### Converting to DataLoader

In [8]:
class Dataset:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [9]:
train = Dataset(X_train, y_train)
test = Dataset(X_test, y_test)
len(train), len(test)

(8000, 2000)

In [10]:
train_dataloader = DataLoader(train, bs, True)
test_dataloader = DataLoader(test, bs, True)

# Creating Model

In [29]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 22* 22, 800)
        self.fc2 = nn.Linear(800, 100)
        self.fc3 = nn.Linear(100, 10)


    def forward(self, x):
        # [800, 3, 100, 100])
        x = self.pool(F.relu(self.conv1(x)))  # [800, 6, 48, 48])
        x = self.pool(F.relu(self.conv2(x)))  # [800, 16, 22, 22])
        x = x.view(-1, 16*22*22)        
        x = F.relu(self.fc1(x))               
        x = F.relu(self.fc2(x))               
        x = self.fc3(x)                       
        return x 


model = CNN().to(device)
model = torch.compile(model)

model

OptimizedModule(
  (_orig_mod): CNN(
    (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (fc1): Linear(in_features=7744, out_features=800, bias=True)
    (fc2): Linear(in_features=800, out_features=100, bias=True)
    (fc3): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [30]:
lossCat = nn.CrossEntropyLoss().to(device)
optimiser = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training

In [31]:

for epoch in range(300):
    for step, (x,y) in enumerate(train_dataloader):
        # forward pass
        x = x.reshape(-1, 3, imgx, imgy)
        y_pred = model(x)
        
        # Loss calculation
        loss = lossCat(y_pred, y)
        
        # Backpropogation
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()
    
    if epoch%10 == 0:    
        print('Epoch:', epoch, ':', loss.item())
    if loss.item() < 1: break # to prevent overfitting

Epoch: 0 : 3.5323781967163086
Epoch: 10 : 2.376026153564453
Epoch: 20 : 2.2872822284698486
Epoch: 30 : 2.247490406036377
Epoch: 40 : 2.221935749053955
Epoch: 50 : 2.1968088150024414
Epoch: 60 : 2.170743942260742
Epoch: 70 : 2.143101215362549
Epoch: 80 : 2.113313674926758
Epoch: 90 : 2.0816335678100586
Epoch: 100 : 2.0478100776672363
Epoch: 110 : 2.0123374462127686
Epoch: 120 : 1.9753646850585938
Epoch: 130 : 1.9369629621505737
Epoch: 140 : 1.8971893787384033
Epoch: 150 : 1.856330156326294
Epoch: 160 : 1.8142566680908203
Epoch: 170 : 1.7708736658096313
Epoch: 180 : 1.7264783382415771
Epoch: 190 : 1.6812928915023804
Epoch: 200 : 1.635522723197937
Epoch: 210 : 1.5890061855316162
Epoch: 220 : 1.5425472259521484
Epoch: 230 : 1.4967881441116333
Epoch: 240 : 1.4505051374435425
Epoch: 250 : 1.404657006263733
Epoch: 260 : 1.3616228103637695
Epoch: 270 : 1.316123604774475
Epoch: 280 : 1.2723835706710815
Epoch: 290 : 1.2294018268585205


# Model Evaluation

### Train Data

In [32]:
with torch.no_grad():
    tot, correct = 0, 0
    for x,y in train_dataloader:
        x = x.reshape(-1,3, imgx, imgy)
        y_pred = model(x)
        correct += sum(torch.argmax(y_pred,dim=1) == torch.argmax(y,dim=1))
        tot += len(y)

    print("Accuracy:", (correct/tot*100).item(),'%')
    tot
        

Accuracy: 69.73750305175781 %


### Test Data

In [33]:
with torch.no_grad():
    tot, correct = 0, 0
    for x,y in test_dataloader:
        x = x.reshape(-1,3, imgx, imgy)
        y_pred = model(x)
        correct += sum(torch.argmax(y_pred,dim=1) == torch.argmax(y,dim=1))
        tot += len(y)

    print("Accuracy:", (correct/tot*100).item(),'%')
    tot

Accuracy: 26.600000381469727 %
